## Exercise 2c)

In this notebook, we compute the testing MSE with the use of PyTorch, for the same model architectures and gradient descent methods as in the code in the regression analysis exercise 2b. 

**NOTE** One of the authors (Frederik) had trouble with importing torch with the virtual environment used in this project. This worked just fine for the other author (Heine). If you have a computer that runs into a problem with this, then it might help to create a separate virtual environment and install all of the libraries from the requirements file. Doing this always worked. 

In [ ]:
LOAD_FROM_FILE = True   # load data from file, so the bottlenecks of the program run in just a few seconds

In [ ]:
# Imports

import torch
from torch import nn, optim
from torch.utils.data import DataLoader

import numpy as np
import matplotlib.pyplot as plt
from utils.utils import skip_if, generate_dataset
from sklearn.model_selection import train_test_split

plt.style.use('./utils/_plot_style.mplstyle')

In [ ]:
# Use device of the acceleratorif it is available; else, use cpu

device = torch.accelerator.current_accelerator().type if torch.accelerator.is_available() else "cpu"
print(f"Using {device} device")

Using cpu device


In [ ]:
# Generate dataset
np.random.seed(124)

n = 300
x, y = generate_dataset(num=n)
x_train, x_test, y_train, y_test = train_test_split(x, y,test_size=0.2,random_state=44)
num_nodes = 50
num_hidden_layers = [1,2]

# Define training methods
training_methods = [
    "SGD",
    "GD",
]

# Step methods
step_methods = [
    "RMSprop", 
    "ADAM"
]

In [ ]:
# define dataset tensors and loaders from the training and testing data

x_train_tensor = torch.tensor(x_train, dtype=torch.float32).to(device).float()
y_train_tensor = torch.tensor(y_train, dtype=torch.long).to(device).float()
x_test_tensor  = torch.tensor(x_test, dtype=torch.float32).to(device).float()
y_test_tensor  = torch.tensor(y_test, dtype=torch.long).to(device).float()

train_dataset = torch.utils.data.TensorDataset(x_train_tensor, y_train_tensor)
train_loader_SGD = DataLoader(train_dataset, batch_size=60, shuffle=True)  # train_loader for stochastic gradient descent
train_loader_GD = DataLoader(train_dataset, shuffle=False)    # train_loader for plain gradient descent

test_dataset = torch.utils.data.TensorDataset(x_test_tensor, y_test_tensor)
test_loader = DataLoader(test_dataset, shuffle=False)

In [ ]:
# define class for PyTorch neural network

class NeuralNetwork_regression(nn.Module):

    """
    Feedforward neural network with PyTorch.

    Parameters
    ----------
    num_nodes : int
        Number of nodes in each layer. 
    num_hidden_layers: int 
        Number of hidden layers in the model. 
    """

    def __init__(self,num_nodes,num_hidden_layers):
        super().__init__()

        model_layers = [nn.Linear(1,num_nodes),nn.Sigmoid()]
        for i in range(num_hidden_layers-1): 
            model_layers.append(nn.Linear(num_nodes,num_nodes))
            model_layers.append(nn.Sigmoid())
        model_layers.append(nn.Linear(num_nodes,1))

        self.linear_stack = nn.Sequential(*model_layers)

    def forward(self, x):   # forward pass of the model
        out = self.linear_stack(x)
        return out
    
    def testing_MSE(self):  # return testing MSE
        with torch.no_grad():  # disable gradient calculation for evaluation 
            input, target = test_dataset.tensors
            output = self.linear_stack(input)
        return torch.mean((target - output)**2)/len(target)
            
    
    def train_model_regression(self,lr=0.01, num_epochs=3000, lambd=0,training_method = "SGD",step_method = "ADAM"):   # train NN model for regression, and return testing MSE
        criterion = nn.MSELoss(reduction='mean')

        if training_method == "SGD": 
            loader = train_loader_SGD
            n_batches = 5   # define number of batches, so the learning rate can be divided by it (gives correct correspondence with our neural network code)
        else: 
            loader = train_loader_GD
            n_batches = 1

        if step_method == "ADAM":
            optimizer = optim.Adam(self.parameters(), lr=lr/n_batches, weight_decay=lambd) # lambd is regularization parameter for L2 regularization
        if step_method=="RMSprop": 
            optimizer = optim.RMSprop(self.parameters(), alpha = 0.9,lr=lr/n_batches, weight_decay=lambd) # lambd is regularization parameter for L2 regularization

        for _ in range(num_epochs):
            self.train()  # set model to training mode

            for input, target in loader: 
                optimizer.zero_grad()            # reset gradients to zero
                outputs = self.linear_stack(input)          # forward pass: compute predictions
                loss = criterion(outputs,target)  # compute MSE
                loss.backward()                 # backpropagate to compute gradients
                optimizer.step()                # update weights using SGD step 

        return self.testing_MSE()

In [ ]:
# Download the data for the best learning rates for SGD and GD from exercise 2b. 

filepath_1 = "../data/best_learning_rate_SGD_final.npy"
filepath_2 = "../data/best_learning_rate_GD_final.npy"

best_learning_rates_SGD = np.load(filepath_1)
best_learning_rates_GD = np.load(filepath_2)

In [ ]:
%%skip_if LOAD_FROM_FILE

n_learning_rates = 9
num_iterations = 3000
pytorch_mse_data = np.zeros((len(num_hidden_layers), len(training_methods),len(step_methods)))

# Analyze pytorch testing mse vs. number of hidden layers, training methods and step methods 
for i in range(len(num_hidden_layers)):
    print(num_hidden_layers[i])
    for j, training_method_name in enumerate(training_methods):
        for l, step_method_name in enumerate(step_methods): 
            print(".", end="")
            np.random.seed(124)
            torch.manual_seed(124)

            if training_method_name == "SGD": 
                best_lr = best_learning_rates_SGD
            else: 
                best_lr = best_learning_rates_GD

            model = NeuralNetwork_regression(num_nodes, num_hidden_layers[i]).to(device)  # define PyTorch neural network
            mse_data = model.train_model_regression(lr = best_lr[i,l],training_method = training_method_name,step_method = step_method_name)  # find testing MSE
            pytorch_mse_data[i][j][l] = mse_data


In [ ]:
filepath = "../data/regression_mse_data_pytorch.npy"
#np.save(filepath, pytorch_mse_data)
if LOAD_FROM_FILE:
    pytorch_mse_data = np.load(filepath)

In [ ]:
# Print PyTorch testing MSE for SGD and GD

print("PyTorch MSE for SGD: ",pytorch_mse_data[:,0,:]) 
print("PyTorch MSE for GD: ",pytorch_mse_data[:,1,:]) 

PyTorch MSE for SGD:  [[2.00000959e-05 2.80611516e-06]
 [6.76921445e-06 2.63497259e-06]]
PyTorch MSE for GD:  [[4.41806858e-09 5.74487694e-06]
 [3.23084848e-08 4.10083931e-06]]
